**Mount drive and clone repository**

In [ ]:
# Mount drive
from google.colab import drive
drive.mount('/content/drive')

# Clone repository
import sys
!git clone https://github.com/fgiacome/MLDL23-FL-project.git
sys.path.append('./MLDL23-FL-project/')
%cd MLDL23-FL-project
# !git checkout fed_w_avg

**Server implementation**

In [ ]:
import copy
from collections import OrderedDict
from utils.stream_metrics import StreamSegMetrics
import random
import numpy as np
import torch


class Server:
    def __init__(
        self,
        clients_per_round,
        num_rounds,
        epochs_per_round,
        train_clients,
        test_clients,
        model,
        metrics,
        random_state=300890,
    ):
        self.clients_per_round = clients_per_round
        self.num_rounds = num_rounds
        self.train_clients = train_clients
        self.test_clients = test_clients
        self.model = model
        self.epochs_per_round = epochs_per_round
        # self.metrics = {
        #     'Train': StreamSegMetrics(n_classes = 16, name = 'Mean IoU'),
        #     'Validation': StreamSegMetrics(n_classes = 16, name = 'Mean IoU'),
        #     'Test': StreamSegMetrics(n_classes = 16, name = 'Mean IoU')
        # }
        # The model parameters are saved to a dict and loaded from
        # the same dict when it gets updated
        self.model_params_dict = copy.deepcopy(self.model.state_dict())
        # The list of client updates in a round. It is a list of tuples
        # of the form: (training set size, update)
        self.updates = []
        ## This line stays commented for now, plan is
        ## to call random.seed(...) in the notebook explicitly
        ## so as to intuitively restore actually unpredictable behavior
        # self.prng = np.random.default_rng(random_state)
        self.prng = np.random.default_rng()

    def select_clients(self):
        """
        This method selects a random subset of `self.clients_per_round` clients
        from the given traning clients, without replacement.
        :return: list of clients
        """
        num_clients = min(self.clients_per_round, len(self.train_clients))
        return self.prng.choice(self.train_clients, num_clients, replace=False)

    def load_model_on_clients(self):
        """
        This function loads the centralized model to the clients at
        the beginning of each training / testing round.
        """
        for c in self.test_clients + self.train_clients:
            c.model.load_state_dict(self.model_params_dict, strict=False)

    def train_round(self, clients):
        """
        This method trains the model with the dataset of the clients.
        It handles the training at single round level.
        The client updates are saved in the object-level list,
        they will be aggregated.
        :param clients: list of all the clients to train
        """
        train_loss_miou = {str(c): {} for c in clients}

        for i, client in enumerate(clients):
            num_samples = client.get_num_samples()

            # Train the single client model
            loss, miou = client.train(self.epochs_per_round)
            train_loss_miou[str(client)]["Loss"] = loss
            train_loss_miou[str(client)]["mIoU"] = miou

            # Get model parameters
            update = client.generate_update()

            # The list of updates is saved at instance level,
            # but it is also returned as an independent list after each
            # train round.
            self.updates.append((num_samples, update))
        return train_loss_miou

    def aggregate(self):
        """
        This method handles the FedAvg aggregation
        :param updates: updates received from the clients
        :return: aggregated parameters
        """
        # Here we make the average of the updated weights
        total_weight = 0
        base = OrderedDict()
        for client_samples, client_model in self.updates:
            total_weight += client_samples
            for key, value in client_model.items():
                if key in base:
                    base[key] += client_samples * value.type(torch.FloatTensor)
                else:
                    base[key] = client_samples * value.type(torch.FloatTensor)
        # averaged_sol_n = copy.deepcopy(self.model_params_dict)
        for key, value in base.items():
            if total_weight != 0:
                # averaged_sol_n[key] = value.to('cuda') / total_weight
                self.model_params_dict[key] = value.to("cuda") / total_weight

        # self.model.load_state_dict(averaged_sol_n, strict=False)
        self.model.load_state_dict(self.model_params_dict, strict=False)
        # self.model_params_dict = copy.deepcopy(self.model.state_dict())
        self.updates = []

    def train(self):
        """
        This method orchestrates the training the evals and tests at rounds level
        :return: list (one elem per round) of dicts (one key per client) of dicts
            (loss, miou) of lists (one elem per epoch) of scalars
        """
        orchestra_statistics = []
        for r in range(self.num_rounds):
            self.load_model_on_clients()
            clients = self.select_clients()
            train_stats = self.train_round(clients)
            self.aggregate()
            test_stats = self.test()
            orchestra_statistics.append((train_stats, test_stats))
        return orchestra_statistics

    def eval_train(self):
        """
        This method handles the evaluation on the train clients
        :return: dict (one key per client) of dicts (loss, miou) of scalars
        """
        self.load_model_on_clients()
        eval_statistics = {str(c): {} for c in self.train_clients}
        for c in self.train_clients:
            l, m = c.test()
            eval_statistics[str(c)]["Loss"] = l
            eval_statistics[str(c)]["mIoU"] = m
        return eval_statistics

    def test(self):
        """
        This method handles the test on the test clients
        :return: dict (one key per client) of dicts (loss, miou) of scalars
        """
        self.load_model_on_clients()
        eval_statistics = {str(c): {} for c in self.test_clients}
        for c in self.test_clients:
            l, m = c.test()
            eval_statistics[str(c)]["Loss"] = l
            eval_statistics[str(c)]["mIoU"] = m
        return eval_statistics

**Transforms and model importations**

In [ ]:
# New transforms
import PIL
from PIL import Image
import numpy as np
class DegradeGaussianBlur(object):
    """
    Strongly blur the image
    """
    def __init__(self, radius=5):
        self.radius = radius

    def __call__(self, img, lbl=None):
        blurry = img.filter(PIL.ImageFilter.GaussianBlur(radius=self.radius))
        if lbl is not None:
            return blurry, lbl
        else:
            return blurry

class DegradeGaussianNoise(object):
    """
    Insert noise in the image
    """
    def __init__(self, sigma=35):
        self.sigma = sigma

    def __call__(self, img, lbl=None):
        np_img = np.array(img)
        h = np_img.shape[0]
        w = np_img.shape[1]
        noise = np.int8(np.random.normal(size=(h,w,1), scale=self.sigma)
                        .clip(-128,127))
        noisy = np.uint8((noise + np_img).clip(0,255))
        noisy_PIL = Image.fromarray(noisy)
        if lbl is not None:
            return noisy_PIL, lbl
        else:
            return noisy_PIL

class DegradePumpGreen(object):
    """
    Insert noise in the image
    """
    def __init__(self, offset=80, sigma=10):
        self.sigma = sigma
        self.offset = offset

    def __call__(self, img, lbl=None):
        np_img = np.array(img)
        h = np_img.shape[0]
        w = np_img.shape[1]
        noise = np.int8((np.random.normal(size=(h,w,1), scale=self.sigma)+self.offset)
                        .clip(-128,127))
        noise_ch2 = np.pad(noise, ((0,0),(0,0),(1,1)))
        noisy = np.uint8((noise_ch2 + np_img).clip(0,255))
        noisy_PIL = Image.fromarray(noisy)
        if lbl is not None:
            return noisy_PIL, lbl
        else:
            return noisy_PIL

In [ ]:
# Importations
import main
import torch
import random
from datasets.idda import IDDADataset
from models import deeplabv3, mobilenetv2
import datasets.ss_transforms as sstr

# Fix random seed
random.seed(300890)

# args for dataset and model importation
class args:
  dataset = 'idda'
  model = 'deeplabv3_mobilenetv2'

# Model
model = main.model_init(args).cuda()

# Transforms
train_transforms = sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
test_transforms = sstr.Compose([
            sstr.Resize((512, 928)),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
train_transforms_degrade = []
train_transforms_degrade.append(sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            DegradeGaussianBlur(),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]))
train_transforms_degrade.append(sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            DegradeGaussianNoise(),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]))
train_transforms_degrade.append(sstr.Compose([
            sstr.ColorJitter(brightness = (0.55, 1.6), contrast = (0.6, 1.6),
                             saturation = (0.5, 1.6), hue = (-0.05, 0.05)),
            sstr.RandomRotation(degrees = (-5, 5)),
            sstr.PadCenterCrop((925, 1644), fill=255), # this removes the
                                                       # rotation margins
            sstr.Resize((512, 928)),
            DegradePumpGreen(),
            sstr.ToTensor(),
            sstr.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ]))

**Train, Test Same Dom, Test Diff Dom importations**

In [ ]:
# Importations
import os
import json
from collections import OrderedDict

# Initialization of clients datasets
root = 'data/idda'
with open(os.path.join(root, 'train.json'), 'r') as f:
  clients_dict = json.load(f)

# Dict -> datasets
train_clients_df = OrderedDict()
map_clients_transforms = {18: 0, 19: 0, 20: 1, 21: 1, 22: 2, 23: 2}
for i, client_id in enumerate(clients_dict.keys()):
  if i < 18:
    train_clients_df[client_id] = IDDADataset(root = root, list_samples = clients_dict[client_id],
                                             transform = train_transforms, client_name = client_id)
  else:
    train_clients_df[client_id] = IDDADataset(root = root, list_samples = clients_dict[client_id],
                                             transform = train_transforms_degrade[map_clients_transforms[i]], client_name = client_id)
train_clients_names = list(train_clients_df.keys())

# Test set
with open(os.path.join(root, 'test_same_dom.txt'), 'r') as f:
    test_same_dom_data = f.read().splitlines()
    test_same_dom_df = IDDADataset(root = root, list_samples = test_same_dom_data,
                                        transform = test_transforms,
                                        client_name='test_same_dom')

with open(os.path.join(root, 'test_diff_dom.txt'), 'r') as f:
    test_diff_dom_data = f.read().splitlines()
    test_diff_dom_df = IDDADataset(root=root, list_samples = test_diff_dom_data,
                                        transform = test_transforms,
                                        client_name='test_diff_dom')

**Combinations**

In [ ]:
# Combinations to test with the format [n_clients_round, n_local_epochs]
combinations = {'comb_2_1': [2, 1],
                'comb_2_3': [2, 3],
                'comb_2_6': [2, 6],
                'comb_4_1': [4, 1],
                'comb_4_3': [4, 3],
                'comb_4_6': [4, 6],
                'comb_8_1': [8, 1],
                'comb_8_2': [8, 3],
                'comb_8_6': [8, 6]}

**Test cell**

In [ ]:
# Client and Server importations
import copy
from client import Client

# Select combination
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
comb = 'comb_4_6' ##############################################################
################################################################################
############################## CHOOSE COMB! ####################################
################################################################################
################################################################################
# Comb_dict formats: [<n_clients_round>, <n_local_epochs>] #####################
#################### [       0     ,             1       ] #####################
################################################################################

# Assign combinations
clients_per_round = combinations[comb][0]
epochs_per_round = combinations[comb][1]

# Number of total rounds
num_rounds = 20

# !!CHOOSE REDUCTION!!
# reduction \in \{'MeanReduction, HardNegativeMining'\}
reduction = 'MeanReduction'

# Clients initializations
# - Train clients
train_clients = [Client(client_dataset = train_clients_df[name], batch_size = 8,
                 model = main.model_init(args).cuda(), dataloader = 'train',
                 optimizer = 'Adam', lr = 1e-3, device = 'cuda:0',
                 reduction = reduction) for name in train_clients_names]

# - Test clients [same_dom, diff_dom]
test_clients = [Client(client_dataset = test_same_dom_df, batch_size = 16,
                model = main.model_init(args).cuda(), dataloader = 'test',
                device = 'cuda:0', reduction = reduction),
                Client(client_dataset = test_diff_dom_df, batch_size = 16,
                model = main.model_init(args).cuda(), dataloader = 'test',
                device = 'cuda:0', reduction = reduction)]

# Server initialization
server = Server(clients_per_round = clients_per_round, num_rounds = num_rounds,
                epochs_per_round = epochs_per_round, train_clients = train_clients,
                test_clients = test_clients, model = main.model_init(args).cuda(),
                metrics = None, random_state = 300890)

# Train server on selected combination
results = server.train()
eval_res = server.eval_train()
# results_eval = server.eval_train()
# results_test = server.test()

# Cast dict
# Results dict format {<round_index>: <results>}
results_dict = {round_idx: results[round_idx] for round_idx in range(num_rounds)}
results_complete = {
      "Train" : [r[0] for r in results],
      "Test" : [r[1] for r in results],
      "Eval": eval_res
  }

# Save results
# Define file name
filename = 'results_' + comb + '.json'
path = '/content/drive/MyDrive/mldl-project-2b/Results6/'

# Save data on file system (Remember to download it!)
with open(path + filename, 'w') as fp:
  json.dump(results_complete, fp)

In [ ]:
same_dom = [res['test_same_dom']['mIoU']['Mean IoU'] for res in results_complete['Test']]
diff_dom = [res['test_diff_dom']['mIoU']['Mean IoU'] for res in results_complete['Test']]

In [ ]:
import matplotlib.pyplot as plt
plt.plot(same_dom, label = 's')
plt.plot(diff_dom, label = 'd')